# RAG 사용 유무에 따른 피드백 결과 비교

## 1. 피드백

In [30]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage(model="solar-pro")
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 14, 'total_tokens': 24, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'solar-pro-241126', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c42b12bd-09b5-41fc-8e22-86bbb8387c39-0', usage_metadata={'input_tokens': 14, 'output_tokens': 10, 'total_tokens': 24, 'input_token_details': {}, 'output_token_details': {}})

In [31]:
import time

In [36]:
# 예시 텍스트
text = """
별의 형성 성운 단계:성운(가스와 먼지 구름)이 온도 하강으로 인해 수축이 시작.중심부가 차가워지며 밀도가 높아지고, 원시별이 형성됨. 원시별 단계:핵융합 반응이 바로 시작되며 중심부 온도가 약 1천만 K로 상승. 주계열성 단계 핵융합 시작:중심부 온도가 약 10억 K에 도달하면 수소가 헬륨으로 융합되며 에너지를 방출. 특징:별은 내부의 중력과 외부로 나가는 방사압이 균형을 이루지 못하며 진동을 반복.질량이 클수록 수명이 길어지고 더 밝으며 온도는 낮아짐. 거성 또는 초거성 단계 핵융합 종료 후 부풀어오름:중심부의 수소가 모두 소모되면 별 전체가 서서히 수축하며 적색 거성 또는 초거성이 됨. 핵융합 반복:질량이 큰 별은 헬륨 이후에도 탄소, 산소, 규소 등을 융합하며 금까지 생성.
"""

In [ ]:
def provide_feedback(input_text):
    # LLM을 이용해 피드백 요청
    llm = ChatUpstage(api_key=UPSTAGE_API_KEY,model="solar-pro")
    
    prompt = f"""
    다음 사용자가 공부하며 작성한 텍스트에 대해 수능 문제를 푸는 사람을 대상으로 피드백을 작성해주세요.
    잘못된 개념 학습을 방지하기 위해 피드백을 제공하고자 합니다.
    피드백은 틀린 내용, 고칠 내용, 해설 순으로 작성되어야 하며, 피드백이 여러 개 있을 경우 번호를 매기세요.
    해설은 간결하게 제공해주세요.
    텍스트: {input_text}
    """

    # LLM 호출
    response = llm.invoke(prompt)
    
    # 전체 응답을 그대로 반환
    return response.content

start = time.time()
# 피드백 제공
feedback = provide_feedback(text)

# 전체 피드백 출력
print(feedback)

end = time.time()
print(end-start)


    1. 피드백: '별의 형성' 과정에서 '원시별'이 형성될 때, 수축으로 인해 중심부의 온도가 상승하는 것이 맞습니다. 하지만 핵융합 반응이 바로 시작되는 것이 아니라, 중심부 온도가 약 1천만 K에 도달해야 핵융합 반응이 시작됩니다.

    해설: '원시별'은 아직 핵융합 반응이 시작되지 않은 상태로, 중심부 온도가 약 1천만 K에 도달해야 핵융합 반응이 시작됩니다.

    2. 피드백: '주계열성' 단계에서 '특징'으로 '별은 내부의 중력과 외부로 나가는 방사압이 균형을 이루지 못하며 진동을 반복'이라고 설명한 부분은 잘못된 내용입니다. 실제로는 내부의 중력과 외부로 나가는 방사압이 균형을 이루어 안정된 상태를 유지합니다.

    해설: '주계열성' 단계에서는 내부의 중력과 외부로 나가는 방사압이 균형을 이루어 안정된 상태를 유지합니다.

    3. 피드백: '거성 또는 초거성 단계'에서 '질량이 클수록 수명이 길어지고 더 밝으며 온도는 낮아짐'이라고 설명한 부분은 잘못된 내용입니다. 실제로는 질량이 클수록 수명이 짧아지고 더 밝으며 온도는 높아집니다.

    해설: '거성 또는 초거성 단계'에서 질량이 클수록 수명이 짧아지고 더 밝으며 온도는 높아집니다.
6.938148736953735


In [33]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_upstage import ChatUpstage, UpstageEmbeddings
from pinecone import Pinecone, ServerlessSpec

In [34]:
import os

def langsmith(project_name=None, set_enable=True):

    if set_enable:
        result = os.environ.get("LANGCHAIN_API_KEY")
        if result is None or result.strip() == "":
            print("LangChain API Key가 설정되지 않았습니다. 참고: https://wikidocs.net/250954")
            return
        os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"  # LangSmith API 엔드포인트
        os.environ["LANGCHAIN_TRACING_V2"] = "true"  # true: 활성화
        os.environ["LANGCHAIN_PROJECT"] = project_name  # 프로젝트명
        print(f"LangSmith 추적을 시작합니다.\n[프로젝트명]\n{project_name}")
    else:
        os.environ["LANGCHAIN_TRACING_V2"] = "false"  # false: 비활성화
        print("LangSmith 추적을 하지 않습니다.")


In [ ]:
def analysis_chunk(input_data):
    # LangSmith 시작
    langsmith("rag-test")
    db_index_name = "dev-02"

    # Pinecone 연결
    pc = Pinecone()
    index = pc.Index(db_index_name)
    embeddings = UpstageEmbeddings(model="embedding-query")
    vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

    # 검색기(Retriever) 생성
    retriever = vectorstore.as_retriever()

    # 입력 데이터와 관련된 정보 검색
    retrieved_docs = retriever.invoke(input_data)
    doc = retrieved_docs[0]
    doc_context = doc.page_content
    rag_id = doc.id

    # 프롬프트 템플릿 생성 (피드백 형식)
    prompt = PromptTemplate.from_template(
        """
        다음 사용자가 공부하며 작성한 텍스트에 대해 수능 문제를 푸는 사람을 대상으로 피드백을 작성해주세요.
        잘못된 개념 학습을 방지하기 위해 피드백을 제공하고자 합니다.
        피드백은 틀린 내용, 고칠 내용, 해설 순으로 작성되어야 하며, 피드백이 여러 개 있을 경우 번호를 매기세요.
        해설은 간결하게 제공해주세요.
    

        #정보:
        {context}

        #입력:
        {question}

        #답:"""
    )
    
    # 언어모델(LLM) 생성
    llm = ChatUpstage(model="solar-pro")
    
    # RAG: 문서 검색 후 피드백 생성
    chain = {"context": RunnablePassthrough(), "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()
    
    # RAG 체인 실행
    response = chain.invoke({"context": doc_context, "question": input_data})

    # 결과 반환
    return {
        "rag_id": rag_id,
        "response": response,
    }

start = time.time()
# 피드백 제공
feedback = analysis_chunk(text)

# 전체 피드백 출력
print(feedback['response'])
end = time.time()
print(f'time: {end-start}')

LangSmith 추적을 시작합니다.
[프로젝트명]
tag-test

1. 잘못된 내용: "중심부 온도가 약 1천만 K로 상승."
   고칠 내용: "중심부 온도가 약 1천만 K까지 상승."
   해설: 원시별 단계에서 중심부 온도가 상승하는 과정은 점진적으로 일어나며, 약 1천만 K에 도달하는 것은 오랜 시간에 걸쳐 이루어집니다. 따라서 "약 1천만 K로 상승" 대신 "약 1천만 K까지 상승"이라고 표현해야 합니다.

2. 잘못된 내용: "별은 내부의 중력과 외부로 나가는 방사압이 균형을 이루지 못하며 진동을 반복."
   고칠 내용: "별은 내부의 중력과 외부로 나가는 방사압이 균형을 이루지 못하며 주기적인 팽창과 수축을 반복."
   해설: 별의 내부에서 일어나는 중력과 방사압의 불균형은 별이 진동을 반복하는 것이 아니라 주기적인 팽창과 수축을 하는 단계로 이어집니다. 이를 주계열성의 진화 과정이라고 합니다. 따라서 "진동을 반복" 대신 "주기적인 팽창과 수축을 반복"이라고 표현해야 합니다.

3. 잘못된 내용: "질량이 클수록 수명이 길어지고 더 밝으며 온도는 낮아짐."
   고칠 내용: "질량이 클수록 수명이 짧아지고 더 밝으며 표면 온도는 낮아짐."
   해설: 별의 질량이 클수록 핵융합 반응이 더 빠르게 일어나며, 이는 별의 수명을 단축시킵니다. 그러나 질량이 큰 별은 더 밝게 빛나며, 표면 온도는 낮습니다. 따라서 "질량이 클수록 수명이 길어져" 대신 "질량이 클수록 수명이 짧아져"라고 표현해야 하며, "온도는 낮아짐" 대신 "표면 온도는 낮아짐"이라고 표현해야 합니다.
time: 11.413607835769653


## 2. 퀴즈

In [37]:
def provide_feedback(input_text):
    # LLM을 이용해 피드백 요청
    llm = ChatUpstage(api_key=UPSTAGE_API_KEY,model="solar-pro")
    
    prompt = f"""
    아래 텍스트를 기반으로 수능 수준의 O/X 퀴즈 5개를 만들어주세요. 
    다음 조건을 충족해야 합니다:
    1. 질문은 개념 이해와 문제 해결 능력을 평가할 수 있어야 합니다.
    2. 정답이 "O"인 문제와 "X"인 문제의 비율은 균형 있게 구성해주세요.
    3. 질문의 난이도는 수능 수준에 맞춰 구체적이고 사고를 요하는 내용을 포함해야 합니다.
    4. 각 질문의 정답에 대한 설명은 간결하지만 충분히 납득 가능하게 작성해주세요.
    
    텍스트: {input_text}
    """

    # LLM 호출
    response = llm.invoke(prompt)
    
    # 전체 응답을 그대로 반환
    return response.content

start = time.time()
# 피드백 제공
feedback = provide_feedback(text)

# 전체 피드백 출력
print(feedback)

end = time.time()
print(end-start)

1. 별의 형성 과정에서 성운 단계가 핵융합 반응이 시작되는 단계인가요? (X)
   설명: 성운 단계는 온도 하강으로 인해 수축이 시작되는 단계이며, 핵융합 반응이 바로 시작되는 것은 원시별 단계이다.

2. 주계열성 단계에서 별은 내부의 중력과 외부로 나가는 방사압이 균형을 이루지 못하며 진동을 반복하나요? (O)
   설명: 주계열성 단계에서 별은 내부의 중력과 외부로 나가는 방사압이 균형을 이루지 못하며 진동을 반복한다.

3. 질량이 클수록 별의 수명이 짧아지고 온도는 높아지나요? (X)
   설명: 질량이 클수록 별의 수명이 길어지고 온도는 낮아진다.

4. 거성 또는 초거성 단계에서 중심부의 수소가 모두 소모되면 별 전체가 서서히 수축하며 적색 거성 또는 초거성이 되나요? (O)
   설명: 거성 또는 초거성 단계에서 중심부의 수소가 모두 소모되면 별 전체가 서서히 수축하며 적색 거성 또는 초거성이 된다.

5. 질량이 큰 별은 헬륨 이후에도 탄소, 산소, 규소 등을 융합하며 금까지 생성하지 않나요? (O)
   설명: 질량이 큰 별은 헬륨 이후에도 탄소, 산소, 규소 등을 융합하며 금까지 생성할 수 있다.
7.6500465869903564


In [38]:
def analysis_chunk(input_data):
    # LangSmith 시작
    langsmith("rag-test")
    db_index_name = "dev-02"

    # Pinecone 연결
    pc = Pinecone()
    index = pc.Index(db_index_name)
    embeddings = UpstageEmbeddings(model="embedding-query")
    vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

    # 검색기(Retriever) 생성
    retriever = vectorstore.as_retriever()

    # 입력 데이터와 관련된 정보 검색
    retrieved_docs = retriever.invoke(input_data)
    doc = retrieved_docs[0]
    doc_context = doc.page_content
    rag_id = doc.id

    # 프롬프트 템플릿 생성 (피드백 형식)
    prompt = PromptTemplate.from_template(
        """
        아래 텍스트를 기반으로 수능 수준의 O/X 퀴즈 5개를 만들어주세요. 
        다음 조건을 충족해야 합니다:
        1. 질문은 개념 이해와 문제 해결 능력을 평가할 수 있어야 합니다.
        2. 정답이 "O"인 문제와 "X"인 문제의 비율은 균형 있게 구성해주세요.
        3. 질문의 난이도는 수능 수준에 맞춰 구체적이고 사고를 요하는 내용을 포함해야 합니다.
        4. 각 질문의 정답에 대한 설명은 간결하지만 충분히 납득 가능하게 작성해주세요.
    

        #정보:
        {context}

        #입력:
        {question}

        #답:"""
    )
    
    # 언어모델(LLM) 생성
    llm = ChatUpstage(model="solar-pro")
    
    # RAG: 문서 검색 후 피드백 생성
    chain = {"context": RunnablePassthrough(), "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()
    
    # RAG 체인 실행
    response = chain.invoke({"context": doc_context, "question": input_data})

    # 결과 반환
    return {
        "rag_id": rag_id,
        "response": response,
    }

start = time.time()
# 피드백 제공
feedback = analysis_chunk(text)

# 전체 피드백 출력
print(feedback['response'])
end = time.time()
print(f'time: {end-start}')

LangSmith 추적을 시작합니다.
[프로젝트명]
tag-test
1. 태양계가 형성되는 과정에서 성운이 수축하면서 회전하는 것이 중요한 역할을 한다. (O/X)
   정답: O
   설명: 성운이 수축하면서 회전하는 과정에서 밀도 차이가 발생하고, 중심부가 차가워지며 밀도가 높아져 원시별이 형성된다.

2. 태양계의 형성 과정에서 초신성 폭발은 성운에 충격을 주어 밀도 차이를 발생시키지 않는다. (O/X)
   정답: X
   설명: 초신성 폭발은 성운에 충격을 주어 밀도 차이를 발생시키고, 밀도가 높은 부분이 자체 중력으로 수축하여 별이 형성되는 과정을 촉진한다.

3. 별의 형성 과정에서 원시별 단계는 핵융합 반응이 바로 시작되며 중심부 온도가 약 1천만 K로 상승하는 단계이다. (O/X)
   정답: O
   설명: 원시별 단계는 핵융합 반응이 바로 시작되며 중심부 온도가 약 1천만 K로 상승하는 단계이다.

4. 주계열성 단계에서 별은 내부의 중력과 외부로 나가는 방사압이 균형을 이루며 진동을 반복한다. (O/X)
   정답: X
   설명: 주계열성 단계에서 별은 내부의 중력과 외부로 나가는 방사압이 균형을 이룬다. 진동은 거성 또는 초거성 단계에서 발생한다.

5. 핵융합 종료 후 부풀어오르는 거성 또는 초거성 단계에서 질량이 큰 별은 헬륨 이후에도 탄소, 산소, 규소 등을 융합하며 금까지 생성한다. (O/X)
   정답: O
   설명: 핵융합 종료 후 부풀어오르는 거성 또는 초거성 단계에서 질량이 큰 별은 헬륨 이후에도 탄소, 산소, 규소 등을 융합하며 금까지 생성한다.
time: 12.73697280883789
